In [1]:
import openai
import time

client = openai.Client()

vector_store = client.beta.vector_stores.create(name="Tutor de Apostilas")

file = ["data_sample/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id = vector_store.id,
  files=file_stream
)

In [2]:
assistant = client.beta.assistants.create(
  name="Tutor Apostila",
  instructions="Você é um tutor especializado em tecnologias emergentes. \
    Você sabe responder perguntas sobre LLMs como OpenAI, HuggingFace, Gemini, etc. \
    Caso você não encontre as respostas, seja sincero e fale que não sabe responder",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids":[vector_store.id]}},
  model="gpt-4-turbo-preview"
)

In [8]:
question = "Conforme o documento, o que é o Hugging Face e como a empres Gekar tem utilizado ela?"

# Criação da thread
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=question
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Nome do usuário Premium"
)

# Aguarda thread rodar
while run.status in ['queued', "in_progress", "cancelling"]:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Verifica a resposta
if run.status == "completed":
  msgs = client.beta.threads.messages.list(thread_id=thread.id)
  print(msgs)
else: print(f"Error {run.status}")

print(msgs.data[0].content[0].text.value)

SyncCursorPage[Message](data=[Message(id='msg_6UOAq5UrlM9JJAI5kp1btNJf', assistant_id='asst_nPJ7RQTHY468WqvGQpRLd37L', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=657, file_citation=FileCitation(file_id='file-HCt9HngfKnouSrBJdkLvAC'), start_index=644, text='【4:0†LLM.pdf】', type='file_citation')], value='O Hugging Face é destacado como uma comunidade de código aberto que congrega centenas de milhares de modelos de contribuidores que podem auxiliar na resolução de muitos casos de uso específicos, tais como geração de texto, resumo e classificação. Tais comunidades estão rapidamente se aproximando do desempenho de modelos proprietários, embora ainda não tenham conseguido igualar o desempenho de modelos avançados como o GPT-4. Contudo, a comunidade de código aberto exige um pouco mais de esforço para pegar um modelo e começar a utilizá-lo, embora avanços estejam acontecendo rapidamente para tornar esses modelos mais a